In [762]:
# Set up and load modules nothing special here
%matplotlib inline
%load_ext autoreload
%autoreload 2
%load_ext autotime

import seaborn as sns
sns.set(color_codes=True)
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams['savefig.dpi'] = 150
from uvnn.snns.multi_neuron import simulate

#### Generate 4 spike trains, first and third changes periods 10, 40, 10 ... , second and last 40, 10, ... each period cycles  lasts for frag_time seconds

In [837]:
sim_time = 2000
frag_time = 200
def gen_spike(periods = [10, 40], sim_time = 200, frag_time=100):
    # build a spike train with alternating periods
    ret = []
    t_now = 0
    cur_period = 0
    while (t_now < sim_time):
        fragment = range(t_now, t_now + frag_time, periods[cur_period])
        ret.extend(fragment)
        cur_period = (cur_period + 1) % len(periods)        
        t_now = fragment[-1] + periods[cur_period]
    return ret

train1 = gen_spike([50, 83], sim_time, frag_time)
train2 = gen_spike([83, 50], sim_time, frag_time)
#train2 = np.array(train1) + 25
train3 = []
train4 = []
#train3 = gen_spike([5, 21], sim_time, frag_time)
#train4 = gen_spike([21, 5], sim_time, frag_time)
spike_trains = [train1, train2]#, train3, train4]
print train1


In [851]:
conf = {'time_step': 1, # simulation step second
       'simulation_time':sim_time, # seconds
        'D':0.01, # potential degradation value
        'p_max':3,
        'p_min':-2,
        'p_rest':0,
        't_refr':14,
        'n_input':4,
        'n_output':2,
        'spike_trains':spike_trains,
        'w_max':15,
        'w_min':-15,
        'sigma':0.1,
        'margin':0,
        'windowsize':10,
        'tau_plus':20,
        'tau_minus':20,
        'a_plus':0.2,
        'a_minus':-0.2,
        'd_neighbours':0.1
       }
#traces, out_spikes = simulate(**conf)

In [852]:
traces, out_spikes, w_trace = simulate(**conf)
trace1, trace2 = zip(*traces)
print len(train1)
#print len(trace1)
#print trace2
#print out_spikes

In [853]:
plt.rcParams['savefig.dpi'] = 80
w_graph = [np.sum(x) for x in w_trace]
plt.plot(w_graph)

In [854]:
plt.rcParams['savefig.dpi'] = 150
f, axarr = plt.subplots(3, sharex=True)
axarr[0].plot(trace1)
#axarr[0]
axarr[0].set_title('Membrane potentials')
axarr[1].plot(trace2, c='b')

# inputs

def ys(arr, value):
    # get the array with the same length as arr all element value
    return [value for _ in range(len(arr))]


def plot_things(axx):
    arr_inp0 = list(set(train1) | set(train3))

    arr_inp1 = list(set(train2) | set(train4))
    axx.scatter(arr_inp0, ys(arr_inp0, 0), c='r', alpha=1)
    axx.scatter(arr_inp1, ys(arr_inp1, 1), c='k')

    # outputs
    arr_out0 = list(out_spikes[0])
    arr_out1 = list(out_spikes[1])
    axx.scatter(arr_out0, ys(arr_out0, 3), c='y', marker='s')
    axx.scatter(arr_out1, ys(arr_out1, 2), c='g', marker='s')

plot_things(axarr[0])
plot_things(axarr[1])
